In [1]:
import vk_api
import os
import pymorphy2
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
morph = pymorphy2.MorphAnalyzer()
import numpy as np
import tqdm
import tensorflow as tf
from nltk.tokenize import RegexpTokenizer

In [2]:
# numpy
import numpy

# random
from random import shuffle

In [3]:
VK_TOKEN = "2be03a7932fccb910597649a6a9b2609f1931d809c6f4278c0ed1d50f6774ecd792611d900206b10dcdb1"
FB_TOKEN = "EAABtRlqbz5YBAFuHv18RyKN8ZCJiMZARqzPsfCADT4eZApjulZA9hxcBt0vaILFY9jRfgO5KX6ZBsM27xUn3kMmFQQXcGpCYFLjEnVZB3ObRN4dZAXaiOOK0OyJogthersprvYNrzkncCKwoDS7116DRIwm8a9Dead7Ef0gVpxa8m5vxSTrcCRt9IQbGMs8EE8VqM9kZANo6bAZDZD"

In [4]:
def get_posts_fb(user='BillGates'):

    # You'll need an access token here to do anything.  You can get a temporary one
    # here: https://developers.facebook.com/tools/explorer/
    path = f"corpora_from_fb_users/{user}.txt"
    if not os.path.exists(path):
        access_token = FB_TOKEN

        graph = facebook.GraphAPI(access_token)
        profile = graph.get_object(user)
        posts = graph.get_connections(profile['id'], 'posts')
        
        with open(path, 'w') as f:
            texts = []
            while True:
                try:
                    # Perform some action on each post in the collection we receive from
                    # Facebook.
                    for post in posts['data']:
                        msg = post.get('message', '')
                        if msg:
                            processed_text = full_process(msg)
                            texts.append(processed_text)
                            _ = f.write(f"{processed_text}\n")
                    # Attempt to make a request to the next page of data, if it exists.
                    posts = requests.get(posts['paging']['next']).json()
                except KeyError:
                    # When there are no more pages (['paging']['next']), break from the
                    # loop and end the script.
                    break
        return texts
    else:
        with open(path) as f:
            return list(f)
        
def get_posts_fb_temp(user='BillGates'):
    path = f"corpora_from_fb_users/{user}.txt"
    if not os.path.exists(path):
        with open(path, 'w') as f:
            texts = []
            for post in t[user]:
                if post:
                    _ = f.write(f"{post}\n")

In [10]:
def getallwall(kwargs, n=None):
    '''Get all texts from wall generator'''
    vk_session = vk_api.VkApi(token=VK_TOKEN)
    tools = vk_api.VkTools(vk_session)
    if n:
        try:
            wall_posts = tools.get_all_iter("wall.get", 75, values=kwargs, limit=n)
            for i, post in enumerate(wall_posts, 1):
                if i > n:
                    break
                yield post['text']
        except:
            wall_posts = tools.get_all_iter("wall.get", 15, values=kwargs, limit=n)
            for i, post in enumerate(wall_posts, 1):
                if i > n:
                    break
                yield post['text']
    else:
        try:
            wall_posts = tools.get_all_iter("wall.get", 75, values=kwargs)
            for post in wall_posts:
                yield post['text']
        except:
            wall_posts = tools.get_all_iter("wall.get", 15, values=kwargs)
            for post in wall_posts:
                yield post['text']

def get_posts_vk_user(user, n_wall=None):
    path = f"corpora_from_vk_users/{user}.txt"
    if not os.path.exists(path):
        wall = getallwall({"owner_id": user}, n_wall)
        with open(path, "w") as f:
            for post in wall:
                processed_text = full_process(post)
                if processed_text:
                    texts.append(processed_text)
                    _ = f.write(f"{processed_text}\n")
        return texts
    else:
        with open(path) as f:
            return list(f)
        
def process_public_vk(public_id, n_wall=None):
    texts = []
    path = f"corpora_from_vk_publics/{public_id}.txt"
    if not os.path.exists(path):
        wall = getallwall({"owner_id": -public_id}, n_wall)
        with open(path, "w") as f:
            for post in wall:
                processed_text = full_process(post)
                if processed_text:
                    texts.append(processed_text)
                    _ = f.write(f"{processed_text}\n")
        return texts
    else:
        with open(path) as f:
            return list(f)
        
def get_publics(user_id, num_publics):
    vk_session = vk_api.VkApi(token=VK_TOKEN)
    vk = vk_session.get_api()
    groups = vk.groups.get(user_id=user_id, extended=1, fields='members_count', count=25)['items']
    return [g['id'] for g in groups if g.get('members_count', 1000000) < 1000000][:num_publics]

In [6]:
def get_all_links(url):
    base_url = "/".join(url.split("/")[:3])
    
    def check_valid(urls):
        '''Only full links containing base url'''
        base_url_name = base_url.split("/")[-1].split(".")[0]
        valid = list(filter(lambda x: base_url_name in x, urls))
        wo_base_url = list(filter(lambda x: base_url_name not in x and "http" not in x, urls))
        ext = []
        for link in wo_base_url:
            if link.startswith("/"):
                ext.append(base_url + link)
            else:
                ext.append(f"{base_url}/{link}")
        return valid + ext

    def recursive_url(url):
        '''Recursively finds the urls'''
        try:
            page = requests.get(url).text
        except:
            page = requests.get(base_url + url).text
        soup = BeautifulSoup(page, "lxml")
        return set([item.get('href', '') for item in soup.find_all('a')])


    def get_links(url):
        page = requests.get(url).text
        soup = BeautifulSoup(page, "lxml")
        links = set([item.get('href', '') for item in soup.find_all('a')])
        new_links = links
        for link in links.copy():
            try:
                new_links.update(recursive_url(link))
            except: pass
        return check_valid(new_links)
    
    return check_valid(get_links(url))

In [11]:
publics = [60114472, 19191317, 29534144, 23482909, 62438886, 81354264, 
           "http://www.stplan.ru/", "http://www.30n.ru/2/1.html", 65974416, 79084019, 
           "http://www.fcpir.ru/", "https://4science.ru/", 67183197, 66233468, 
           30558759, 98643656, 23509868, 56821139, 78860407, 61490488,
           "http://www.nsh.ru/", "http://россельхоз.рф/", 
           "http://be5.biz/upravlenie/gosudarstvennoe_upravlenie.html", 97296142,
           74686342, 79925455, 98643656, 63337812, 37959220, 10933209,
           "http://www.soldiering.ru", "https://voennoe-delo.com",
           "https://www.cbr.ru/sbrfr/archive/fsfr/ffms/ru/legislation/corp_management_study/index.html", 69693893,
           49582956, 72388807, 30713157, 26978036, 69560028, 73537456, 
           29809500, 128350290, 37876217, 3800580
          ]

In [12]:
sources = {'art_test': 'TEST_ART', 'art_train': 'TRAIN_ART',
           'politics_test': 'TEST_POLITICS', 'politics_train': 'TRAIN_POLITICS',
           'finances_test': 'TEST_FINANCES', 'finances_train': 'TRAIN_FINANCES',
           'strateg_management_test': 'TEST_STRATEG_MANAGEMENT', 'strateg_management_train': 'TRAIN_STRATEG_MANAGEMENT',
           'law_test': 'TEST_LAW', 'law_train': 'TRAIN_LAW',
           'elaboration_test': 'TEST_ELABORATION', 'elaboration_train': 'TRAIN_ELABORATION',
           'industry_test': 'TEST_INDUSTRY', 'industry_train': 'TRAIN_INDUSTRY',
           'education_test': 'TEST_EDUCATION', 'education_train': 'TRAIN_EDUCATION',
           'social_business_test': 'TEST_SOCIAL_BUSINESS', 'social_business_train': 'TRAIN_SOCIAL_BUSINESS',
           'public_health_test': 'TEST_PUBLIC_HEALTH', 'public_health_train': 'TRAIN_PUBLIC_HEALTH',
           'agriculture_test': 'TEST_AGRICULTURE', 'agriculture_train': 'TRAIN_AGRICULTURE',
           'government_management_test': 'TEST_GOVERNMENT_MANAGEMENT', 'government_management_train': 'TRAIN_GOVERNMENT_MANAGEMENT',
           'smm_test': 'TEST_SMM', 'smm_train': 'TRAIN_SMM',
           'innovations_test': 'TEST_INNOVATIONS', 'innovations_train': 'TRAIN_INNOVATIONS',
           'safety_test': 'TEST_SAFETY', 'safety_train': 'TRAIN_SAFETY',
           'military_test': 'TEST_MILITARY', 'military_train': 'TRAIN_MILITARY',
           'corporative_management_test': 'TEST_CORPORATIVE_MANAGEMENT', 'corporative_management_train': 'TRAIN_CORPORATIVE_MANAGEMENT',
           'social_safety_test': 'TEST_SOCIAL_SAFETY', 'social_safety_train': 'TRAIN_SOCIAL_SAFETY',
           'building_test': 'TEST_BUILDING', 'building_train': 'TRAIN_BUILDING',
           'entrepreneurship_test': 'TEST_ENTREPRENEURSHIP', 'entrepreneurship_train': 'TRAIN_ENTREPRENEURSHIP',
           'sport_test': 'TEST_SPORT', 'sport_train': 'TRAIN_SPORT',
           'investitions_test': 'TEST_INVESTITIONS', 'investitions_train': 'TRAIN_INVESTITIONS'
          }

In [23]:
# get data for corpora's test
for tag, id_ in zip(list(sources.keys())[::2], publics[::2]):
    path = f"corpora_test/{tag}.txt"
    s = set()
    if not os.path.exists(path):
        with open(path, "w") as f:
            if isinstance(id_, int):
                wall = getallwall({"owner_id": -id_}, 2000)
                for post in tqdm.tqdm(wall):
                    if len(post) and post not in s:
                        s.add(post)
                        _ = f.write(f"{post}\n")
            elif isinstance(id_, str):
                links = np.random.choice(list(get_all_links(id_)), 2000)
                for link in tqdm.tqdm(links):
                    try:
                        page = requests.get(link).text
                        soup = BeautifulSoup(page, "lxml")
                        for text in soup.text.strip().split("\n"):
                            if len(text) and text not in s:
                                s.add(text)
                                _ = f.write(f"{text}\n")
                    except: pass

100%|██████████| 2000/2000 [23:11<00:00,  1.44it/s]  


In [25]:
# get data for corpora's train
for tag, id_ in zip(list(sources.keys())[1::2], publics[1::2]):
    path = f"corpora_train/{tag}.txt"
    s = set()
    if not os.path.exists(path):
        with open(path, "w") as f:
            if isinstance(id_, int):
                wall = getallwall({"owner_id": -id_}, 2000)
                for post in tqdm.tqdm(wall):
                    if len(post) and post not in s:
                        s.add(post)
                        _ = f.write(f"{post}\n")
            elif isinstance(id_, str):
                links = np.random.choice(list(get_all_links(id_)), 2000)
                for link in tqdm.tqdm(links):
                    try:
                        page = requests.get(link).text
                        soup = BeautifulSoup(page, "lxml")
                        for text in soup.text.strip().split("\n"):
                            if len(text) and text not in s:
                                s.add(text)
                                _ = f.write(f"{text}\n")
                    except: pass

2000it [00:16, 124.23it/s]
2000it [00:13, 145.49it/s]
2000it [00:11, 174.39it/s]
100%|██████████| 2000/2000 [09:47<00:00,  1.68it/s]
1754it [00:07, 225.30it/s]
100%|██████████| 2000/2000 [25:16<00:00,  1.29it/s]  
1976it [00:13, 150.12it/s]
1724it [00:06, 256.14it/s]
1449it [00:10, 140.87it/s]
2000it [00:16, 124.86it/s]
100%|██████████| 2000/2000 [06:45<00:00,  4.93it/s]  
740it [00:04, 174.93it/s]
2000it [00:08, 245.26it/s]
2000it [00:07, 261.04it/s]
2000it [00:07, 256.32it/s]
100%|██████████| 2000/2000 [05:11<00:00,  5.02it/s]
1008it [00:04, 234.19it/s]
669it [00:02, 287.75it/s]
2000it [00:11, 178.09it/s]
2000it [00:16, 120.10it/s]
2000it [00:09, 220.96it/s]
2000it [00:09, 202.54it/s]


In [6]:
ordered_sources = list(sources.items())